<a href="https://colab.research.google.com/github/mihhwiNick/DA_project/blob/main/Benh_tieu_duong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sử dụng thuật toán ADAP để dự đoán bệnh tiểu đường

##1.Định nghĩa vấn đề
+ **Mô tả**:
   + Bộ dữ liệu gồm 768 mẫu từ bệnh nhân đều là nữ giới, ít nhất 21 tuổi, và có nguồn gốc người da đỏ Pima, sống gần Phoenix, Arizona.
   + 8 đặc điểm được đo từ mỗi mẫu + 1 biến phân loại (có/không mắc tiểu đường)
+ **Dữ liệu vào**:
   + Số lần mang thai (lần)
   + Nồng độ glucose huyết tương sau 2 giờ trong xét nghiệm dung nạp glucose đường uống (mg/dL)
   + Huyết áp tâm trương (mm Hg)
   + Độ dày nếp gấp da cơ tam đầu (mm)
   + Nồng độ insulin huyết thanh sau 2 giờ (mu U/ml)
   + Chỉ số khối cơ thể BMI (kg/m²)
   + Hàm phả hệ tiểu đường ()
   + Tuổi (năm)
   + Biến phân loại (nhị phân)
+ **Dữ liệu ra**:
   + Dự đoán bệnh nhân có tiểu đường không (Biến phân loại 0 hoặc 1)

## 2.Chuẩn bị vấn đề

### 2.1.Khai báo thư viện

In [2]:
# Load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.model_selection as mod
import sklearn.neighbors as nei
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report, precision_recall_curve, average_precision_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import operator
%matplotlib inline

### 2.2.Nạp dữ liệu

In [3]:
#load data
pima = pd.read_csv('https://raw.githubusercontent.com/mihhwiNick/DA_project/refs/heads/main/pima-indians-diabetes.data.csv')

## 3.Phân tích dữ liệu

### 3.1.Thống kê mô tả

#### (1) **Hiển thị một số thông tin về dữ liệu**
+ Số dòng, số cột của dữ liệu
+ Kiểu dữ liệu của từng cột
+ 5 dòng đầu và 5 dòng cuối của bảng dữ liệu
+ Thông tin chung về dữ liệu

In [11]:
# Get shape
print(f'+ Shape: {pima.shape}')

# Get types
print(f'\n+ Data Types:\n{pima.dtypes}')

# Check the first 5 rows (head)
print('\n+ First 5 rows:')
display(pima.head(5))

# Check the last 5 rows (tail)
print('\n+ Last 5 rows:')
display(pima.tail(5))

# info
print('\n + Basic Information: ')
pima.info()

+ Shape: (768, 9)

+ Data Types:
Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

+ First 5 rows:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1



+ Last 5 rows:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0



 + Basic Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


** Nhận xét **
+ Dữ liệu có 8 tính chất để phân lớp: Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age
+ Các giá trị đều là số học, thuộc kiểu int64 và float64
+ Tổng số dòng dữ liệu là 768 dòng
+ Dữ liệu phân lớp nằm ở cột Outcome, gồm 2 giá trị: 0 (không mắc bệnh) và 1 (mắc bệnh)


#### (2) Phân loại tình trạng dinh dưỡng theo BMI

In [32]:
# Tạo cột Nutritional Status
Nutritional_status = pd.Series([])

In [33]:
# Nutritional Status dựa trên BMI
for i in range(len(pima)):
    if pima['BMI'][i] == 0.0:
        Nutritional_status[i]="Không có dữ liệu"

    elif pima['BMI'][i] < 18.5:
        Nutritional_status[i]="Thiếu cân"

    elif pima['BMI'][i] < 25:
        Nutritional_status[i]="Bình thường"

    elif pima['BMI'][i] >= 25 and pima['BMI'][i] < 30:
        Nutritional_status[i]="Thừa cân"

    elif pima['BMI'][i] >= 30:
        Nutritional_status[i]="Béo phì"

    else:
        Nutritional_status[i]= pima['BMI'][i]

In [36]:
# Thêm cột Nutritional Status
pima.insert(6, "Nutritional Status", Nutritional_status)

In [37]:
# Kiểm tra dữ liệu sau khi thêm cột mới
pima.head (5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Nutritional Status,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,Béo phì,0.627,50,1
1,1,85,66,29,0,26.6,Thừa cân,0.351,31,0
2,8,183,64,0,0,23.3,Bình thường,0.672,32,1
3,1,89,66,23,94,28.1,Thừa cân,0.167,21,0
4,0,137,40,35,168,43.1,Béo phì,2.288,33,1


In [40]:
print('Bảng phân bố tình trạng dinh dưỡng')
# số lượng trạng thái dinh dưỡng
amount = pima['Nutritional Status'].value_counts()

# Tính phần trăm (dựa trên tổng số mẫu)
percentage = (amount / len(pima) * 100).round(2)

# Tạo DataFrame kết hợp số lượng và phần trăm
bang_ket_qua = pd.DataFrame({
    'Số lượng': amount,
    'Phần trăm (%)': percentage
})
print(bang_ket_qua)

Bảng phân bố tình trạng dinh dưỡng
                    Số lượng  Phần trăm (%)
Nutritional Status                         
Béo phì                  472          61.46
Thừa cân                 179          23.31
Bình thường              102          13.28
Không có dữ liệu          11           1.43
Thiếu cân                  4           0.52


**Nhận xét**:
+ Nhóm "Béo phì" chiếm tỷ lệ áp đảo (61.46%) trong dân số Pima, trong khi nhóm "Thiếu cân" chỉ chiếm 0.52%, phản ánh xu hướng thừa cân rõ rệt trong cộng đồng này.
+ Trong tổng số 768 phụ nữ, có 11 người không có thông tin về BMI. Chỉ 102 người có cân nặng bình thường và 4 người thuộc nhóm thiếu cân; phần lớn còn lại thuộc nhóm thừa cân hoặc béo phì, phản ánh xu hướng dinh dưỡng mất cân đối và cho thấy mối liên hệ tiềm năng giữa tình trạng dinh dưỡng và nguy cơ mắc tiểu đường.

#### (3) Kiểm tra tính toàn vẹn của dữ liệu
+ Dữ liệu có bị trùng lặp không? Hiển thị dòng bị vi phạm.
+ Dữ liệu có tồn tại giá trị Null không? Hiển thị dòng bị vi phạm.
+ Dữ liệu có tồn tại giá trị NaN không? Hiển thị dòng bị vi phạm.

In [17]:
has_null = pima.isnull().sum().any()
has_nan  = pima.isna().sum().any()
n_duplicated = pima.duplicated().sum()

print('Tính toàn vẹn dữ liệu:')
print(f'+ Có giá trị Null: {has_null}')
if has_null:
    display(pima[pima.isnull().any(axis=1)])

print(f'+ Có giá trị NaN: {has_nan}')
if has_nan:
    display(pima[pima.isna().any(axis=1)])

print(f'+ Số dòng trùng: {n_duplicated}')
if n_duplicated > 0:
    # Ví dụ: kiểm tra dòng trùng cụ thể
    s_query1 = "`Pregnancies`==6 and `Glucose`==148 and `Outcome`==1"
    s_query2 = "`Pregnancies`==1 and `Glucose`==85 and `Outcome`==0"
    display(pima.query(s_query1))
    display(pima.query(s_query2))
    display(pima[pima.duplicated()])

Tính toàn vẹn dữ liệu:
+ Có giá trị Null: False
+ Có giá trị NaN: False
+ Số dòng trùng: 0


**Nhận xét**:
+ Dữ liệu không có dòng bị trùng
+ Dữ liệu không có giá trị rỗng (NaN, Null)